# ✅  1 — Objectifs

## 1. Récupération des données Enedis :
   - Source : Récupérer les données de consommation électrique annuelle par adresse depuis l’API Enedis Open Data.
   - Portée : Couvrir toutes les communes de France, en utilisant leur code INSEE (code_commune).
   - Méthode :
      - Requêtes paginées pour chaque commune.
      - Gestion des limites de l’API (rate limiting, erreurs 429/400/5XX).
## 2. Optimisation des requêtes API :
   - Performance :
      - Utiliser des requêtes asynchrones (aiohttp + asyncio) pour réduire le temps total de téléchargement.
      - Limiter le nombre de requêtes simultanées (Semaphore) pour éviter de surcharger l’API.
   - Robustesse :
      - Gérer les erreurs HTTP (ex: 429 Too Many Requests, 502 Bad Gateway).
      - Implémenter des réessais automatiques pour les erreurs temporaires.
      - Respecter les délais entre requêtes (SLEEP_BETWEEN_ENEDIS).

## 3. Sauvegarde incrémentale des données :
   - Format : Stocker les données au format Parquet (compression zstd).
   - Organisation :
      - Découper les données en fichiers par pages/communes (enedis_part_XXXXXXXX.parquet).
      - Centraliser les fichiers dans le dossier parquet_enedis/.
## 4. Système de reprise (Checkpoint) :
   - Fiabilité :
      - Sauvegarder l’état du téléchargement dans un fichier JSON (progress.json).
      - Permettre une reprise automatique en cas d’interruption (crash, arrêt manuel, etc.).
   - Contenu du checkpoint :
      - Dernier département/commune/page traitée.
      - Compteur de fichiers et de lignes pour éviter les doublons.


# ✅  2 — Imports & chemins

In [10]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from config import *

PROJECT_ROOT = Path.cwd().parent

# COMMUNES_CSV_PATH = r"C:\Users\Stagiaire\Desktop\dpe\01-importv_commune_2025.csv"

PARQUET_ENEDIS.mkdir(exist_ok=True, parents=True)

# ✅  3 : Fonctions utilitaires pour le checkpoint

In [1]:
def load_checkpoint_enedis():
    """
    Charge le checkpoint Enedis s'il existe.
    Le checkpoint stocke 'next_page' (page suivante à récupérer).
    """
    if CHECKPOINT_ENEDIS.exists():
        with open(CHECKPOINT_ENEDIS, "r") as f:
            return json.load(f)

    return {
        "current_dep": None,
        "current_commune": None,
        "next_page": 1,
        "last_file_id": 0,
        "total_rows": 0
    }


def save_checkpoint_enedis(dep, commune, next_page, file_id, total_rows):
    checkpoint = {
        "current_dep": dep,
        "current_commune": commune,
        "next_page": int(next_page),
        "last_file_id": int(file_id),
        "total_rows": int(total_rows)
    }
    with open(CHECKPOINT_ENEDIS, "w") as f:
        json.dump(checkpoint, f)

# ✅  4 : Fonction asynchrone pour récupérer les données

In [2]:
async def fetch_data_enedis(session, url, params, headers):
    """
    Requête API robuste avec:
    - retries + backoff sur 429 / erreurs réseau / 5xx
    - timeout total
    Retourne dict JSON ou None.
    """
    backoff = 1.0

    for attempt in range(1, MAX_RETRIES_ENEDIS + 1):
        try:
            timeout = aiohttp.ClientTimeout(total=TIMEOUT_TOTAL_ENEDIS)
            async with session.get(url, params=params, headers=headers, timeout=timeout) as response:

                if response.status == 429:
                    wait = min(SLEEP_ON_429_ENEDIS * backoff, MAX_BACKOFF_ENEDIS)
                    print(f"⚠️ 429 Too Many Requests → pause {wait:.1f}s (tentative {attempt}/{MAX_RETRIES_ENEDIS})")
                    await asyncio.sleep(wait)
                    backoff = min(backoff * BACKOFF_MULTIPLIER_ENEDIS, MAX_BACKOFF_ENEDIS)
                    continue

                if 500 <= response.status < 600:
                    wait = min(backoff + random.random(), MAX_BACKOFF_ENEDIS)
                    print(f"⚠️ HTTP {response.status} serveur → retry dans {wait:.1f}s")
                    await asyncio.sleep(wait)
                    backoff = min(backoff * 1.6, MAX_BACKOFF_ENEDIS)
                    continue

                if response.status != 200:
                    text = await response.text()
                    print(f"⚠️ HTTP {response.status} client. Params={params}. Body={text[:200]}")
                    return None

                content_type = response.headers.get("Content-Type", "")
                if "application/json" not in content_type:
                    text = await response.text()
                    print(f"⚠️ Réponse non-JSON. Content-Type={content_type}. Extrait={text[:200]}")
                    return None

                data = await response.json()
                return data

        except (aiohttp.ClientError, asyncio.TimeoutError) as e:
            wait = min(backoff + random.random(), MAX_BACKOFF_ENEDIS)
            print(f"⚠️ Erreur réseau/timeout: {e} → retry dans {wait:.1f}s (tentative {attempt}/{MAX_RETRIES_ENEDIS})")
            await asyncio.sleep(wait)
            backoff = min(backoff * 1.6, MAX_BACKOFF_ENEDIS)

    print(f"❌ Max retries atteint → abandon requête. Params={params}")
    return None

# ✅  5 : Fonction pour traiter une commune

In [4]:
async def process_commune_enedis(session, dep, commune, file_id, total_rows, start_page=1):
    """
    Télécharge toutes les pages Enedis pour une commune donnée.
    Sauvegarde chaque page en Parquet.
    Met à jour le checkpoint après chaque page écrite.
    """
    page = int(start_page)

    while True:
        params = {
            "size": SIZE_ENEDIS,
            "page": page,
            "q": f"code_commune:{commune}"
        }

        data = await fetch_data_enedis(session, URL_ENEDIS, params, HEADERS_ENEDIS)

        if not data:
            print(f"  {dep}/{commune} : arrêt (pas de data) à la page {page}")
            break

        rows = data.get("results") or []
        if len(rows) == 0:
            print(f"  {dep}/{commune} : fin (page vide) à la page {page}")
            break

        df = pd.DataFrame(rows)

        # Optionnel: réduire aux colonnes utiles si dispo
        if ENEDIS_KEEP_COLUMNS is not None and len(ENEDIS_KEEP_COLUMNS) > 0:
            cols = [c for c in ENEDIS_KEEP_COLUMNS if c in df.columns]
            if len(cols) > 0:
                df = df[cols]

        table = pa.Table.from_pandas(df, preserve_index=False)
        out_file = PARQUET_ENEDIS / f"enedis_part_{file_id:08d}.parquet"
        pq.write_table(table, out_file, compression="zstd")

        n = len(df)
        print(f"  {dep}/{commune} page {page} → {n} lignes → {out_file.name}")

        total_rows += n
        file_id += 1

        # checkpoint = next page
        save_checkpoint_enedis(dep, commune, page + 1, file_id, total_rows)

        page += 1
        await asyncio.sleep(SLEEP_BETWEEN_ENEDIS)

    return file_id, total_rows

# ✅  6 : Boucle principale asynchrone

In [5]:
async def main_enedis():
    communes_par_dep = charger_communes_par_dep()

    checkpoint = load_checkpoint_enedis()
    file_id = checkpoint["last_file_id"]
    total_rows = checkpoint["total_rows"]

    resume = checkpoint["current_dep"] is not None

    print("📌 Reprise activée ?", resume)
    if resume:
        print("📌 Reprise depuis:", checkpoint["current_dep"], checkpoint["current_commune"], "page", checkpoint["next_page"])

    async with aiohttp.ClientSession() as session:
        for dep, communes in communes_par_dep.items():

            if resume and dep != checkpoint["current_dep"]:
                continue

            for commune in communes:
                if resume and dep == checkpoint["current_dep"] and commune != checkpoint["current_commune"]:
                    continue

                start_page = checkpoint["next_page"] if resume else 1

                file_id, total_rows = await process_commune_enedis(
                    session=session,
                    dep=dep,
                    commune=commune,
                    file_id=file_id,
                    total_rows=total_rows,
                    start_page=start_page
                )

                resume = False  # après la première commune reprise

    print("\n✅ EXPORT ENEDIS TERMINÉ")
    print("Fichiers Parquet :", file_id)
    print("Lignes totales :", total_rows)
    print("Checkpoint :", CHECKPOINT_ENEDIS)

# ✅  7 : Exécution et initialisation

In [ ]:
# Exécute la boucle asynchrone
await main_enedis()


📌 Reprise activée ? True
📌 Reprise depuis: 62 62115 page 2
  62/62115 : fin (page vide) à la page 2
  62/62116 page 1 → 6 lignes → enedis_part_00021131.parquet
  62/62116 : fin (page vide) à la page 2
  62/62117 page 1 → 6 lignes → enedis_part_00021132.parquet
  62/62117 : fin (page vide) à la page 2
  62/62118 page 1 → 6 lignes → enedis_part_00021133.parquet
  62/62118 : fin (page vide) à la page 2
  62/62119 page 1 → 1000 lignes → enedis_part_00021134.parquet
  62/62119 page 2 → 295 lignes → enedis_part_00021135.parquet
  62/62119 : fin (page vide) à la page 3
  62/62120 page 1 → 6 lignes → enedis_part_00021136.parquet
  62/62120 : fin (page vide) à la page 2
  62/62121 page 1 → 6 lignes → enedis_part_00021137.parquet
  62/62121 : fin (page vide) à la page 2
  62/62122 page 1 → 6 lignes → enedis_part_00021138.parquet
  62/62122 : fin (page vide) à la page 2
  62/62123 page 1 → 6 lignes → enedis_part_00021139.parquet
  62/62123 : fin (page vide) à la page 2
  62/62124 page 1 → 6 ligne